In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import DepthwiseConv2D
import numpy as np
from flask import Flask, request, render_template, redirect, url_for, flash
from werkzeug.utils import secure_filename
import os

app = Flask(__name__)
app.secret_key = "your_secret_key"

# Path to the saved model
MODEL_PATH = "C:/xampp/htdocs/Agri_IQ/SoilNet_93_86.h5"

# Load the trained model
try:
    SoilNet = load_model(MODEL_PATH)
    print("Model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")

# Define soil classes
classes = {
    0: "Alluvial Soil: { Rice, Wheat, Sugarcane, Maize, Cotton, Soyabean, Jute }",
    1: "Black Soil: { Virginia, Wheat, Jowar, Millets, Linseed, Castor, Sunflower }",
    2: "Clay Soil: { Rice, Lettuce, Chard, Broccoli, Cabbage, Snap Beans }",
    3: "Red Soil: { Cotton, Wheat, Pulses, Millets, Oilseeds, Potatoes }"
}

# Ensure the upload folder exists
UPLOAD_FOLDER = 'static/user_uploaded'
if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)

app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}

# Helper function to check file extension
def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

# Preprocessing and prediction function
def model_predict(image_path, model):
    try:
        image = load_img(image_path, target_size=(224, 224))  # Resize image
        image = img_to_array(image)  # Convert to array
        image = image / 255.0  # Normalize to [0, 1]
        image = np.expand_dims(image, axis=0)  # Add batch dimension

        # Predict the class
        prediction = np.argmax(model.predict(image), axis=1)[0]
        predicted_class = classes.get(prediction)

        # Return soil type and corresponding HTML page
        html_pages = {
            0: "Alluvial.html",
            1: "Black.html",
            2: "Clay.html",
            3: "Red.html"
        }
        return predicted_class, html_pages.get(prediction)
    except Exception as e:
        print(f"Error during prediction: {e}")
        return None, None

# Index route
@app.route('/', methods=['GET'])
def index():
    return render_template('index.html')

# Prediction route
@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Ensure a file was uploaded
        if 'image' not in request.files:
            flash('No file part in the request.')
            return redirect(request.url)

        file = request.files['image']
        
        # Ensure the file has a valid filename
        if file.filename == '':
            flash('No selected file.')
            return redirect(request.url)
        
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)  # Secure the filename
            file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            file.save(file_path)  # Save uploaded file

            # Perform prediction
            pred, output_page = model_predict(file_path, SoilNet)

            if output_page:
                return render_template(output_page, pred_output=pred, user_image=filename)
            else:
                flash("Error during prediction.")
                return redirect(request.url)
        else:
            flash("Invalid file format. Please upload a valid image (png, jpg, jpeg).")
            return redirect(request.url)

    except Exception as e:
        print(f"Error during file processing: {e}")
        return "Error during file processing", 500

# Optional: Clean up uploaded images after use (uncomment if needed)
# def cleanup_upload_folder():
#     for filename in os.listdir(app.config['UPLOAD_FOLDER']):
#         file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
#         if os.path.isfile(file_path):
#             os.remove(file_path)

if __name__ == '__main__':
    app.run(debug=True, threaded=True)


Error loading model: Error when deserializing class 'DepthwiseConv2D' using config={'name': 'expanded_conv_depthwise', 'trainable': False, 'dtype': 'float32', 'kernel_size': [3, 3], 'strides': [1, 1], 'padding': 'same', 'data_format': 'channels_last', 'dilation_rate': [1, 1], 'groups': 1, 'activation': 'linear', 'use_bias': False, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'bias_regularizer': None, 'activity_regularizer': None, 'bias_constraint': None, 'depth_multiplier': 1, 'depthwise_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'depthwise_regularizer': None, 'depthwise_constraint': None}.

Exception encountered: Unrecognized keyword arguments passed to DepthwiseConv2D: {'groups': 1}
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\Ramnivash\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
